In [ ]:

!pip install yfinance
!pip install beautifulsoup4
!pip install pandas
!pip install matplotlib
!pip install lxml

import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests


def make_graph(stock_data, revenue_data, stock, title):
    fig, ax1 = plt.subplots(figsize=(12, 6))

    ax1.plot(stock_data['Date'], stock_data['Close'], color="blue", label='Stock Price')
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Stock Price", color="blue")
    ax1.tick_params(axis='y', labelcolor="blue")
    ax1.set_title(title)

    ax2 = ax1.twinx()
    ax2.plot(revenue_data['Date'], revenue_data['Revenue'], color="red", label='Revenue')
    ax2.set_ylabel("Revenue (in billions)", color="red")
    ax2.tick_params(axis='y', labelcolor="red")

    fig.tight_layout()
    plt.show()

# -------------------------------
# Question 1: Tesla Stock Data
# -------------------------------
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
print("Tesla Stock Data (First 5 rows):")
print(tesla_data.head())

# -------------------------------
# Question 2: Tesla Revenue Data
# -------------------------------
url_tesla = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_tesla = requests.get(url_tesla).text
soup_tesla = BeautifulSoup(html_tesla, "html.parser")
tables = soup_tesla.find_all("table")

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in tables[1].find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace("$", "").replace(",", "")
        if revenue != "":
            tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": float(revenue)}, ignore_index=True)

tesla_revenue["Date"] = pd.to_datetime(tesla_revenue["Date"])
tesla_revenue = tesla_revenue.sort_values("Date")
print("Tesla Revenue Data (Last 5 rows):")
print(tesla_revenue.tail())

# -------------------------------
# Question 3: GME Stock Data
# -------------------------------
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
print("GME Stock Data (First 5 rows):")
print(gme_data.head())

# -------------------------------
# Question 4: GME Revenue Data
# -------------------------------
url_gme = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_gme = requests.get(url_gme).text
soup_gme = BeautifulSoup(html_gme, "html.parser")
tables = soup_gme.find_all("table")

gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in tables[1].find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace("$", "").replace(",", "")
        if revenue != "":
            gme_revenue = gme_revenue.append({"Date": date, "Revenue": float(revenue)}, ignore_index=True)

gme_revenue["Date"] = pd.to_datetime(gme_revenue["Date"])
gme_revenue = gme_revenue.sort_values("Date")
print("GME Revenue Data (Last 5 rows):")
print(gme_revenue.tail())

# -------------------------------
# Question 5: Tesla Dashboard
# -------------------------------
make_graph(tesla_data, tesla_revenue, 'TSLA', 'Tesla Stock Price vs Revenue')

# -------------------------------
# Question 6: GME Dashboard
# -------------------------------
make_graph(gme_data, gme_revenue, 'GME', 'GameStop Stock Price vs Revenue')
